In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from Sinkhorn_CMI import SinkhornImputation_CMI
import sys
import torch

In [2]:
adult_data_train = pd.read_csv(r'C:\Users\admin\Desktop\projects\Imputation_Fairness\Data\adult\adult.data',
                         header=None, 
                        delimiter=',')
adult_data_test= pd.read_csv(r'C:\Users\admin\Desktop\projects\Imputation_Fairness\Data\adult\adult.test',
                         header=None, 
                        delimiter=',', skiprows=1)
adult_data = pd.concat([adult_data_train, adult_data_test], axis=0).reset_index(drop=True)

In [3]:
adult_data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
# Replace all ' ?' values with np.nan
adult_data = adult_data.replace(' ?', np.nan)
adult_data.isna().sum()

0        0
1     2799
2        0
3        0
4        0
5        0
6     2809
7        0
8        0
9        0
10       0
11       0
12       0
13     857
14       0
dtype: int64

In [5]:
missing_percentage = (adult_data.isnull().sum() / len(adult_data)) * 100
print(missing_percentage)

0     0.000000
1     5.730724
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6     5.751198
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.000000
13    1.754637
14    0.000000
dtype: float64


In [6]:
adult_data = adult_data.dropna()

In [7]:
adult_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       45222 non-null  int64 
 1   1       45222 non-null  object
 2   2       45222 non-null  int64 
 3   3       45222 non-null  object
 4   4       45222 non-null  int64 
 5   5       45222 non-null  object
 6   6       45222 non-null  object
 7   7       45222 non-null  object
 8   8       45222 non-null  object
 9   9       45222 non-null  object
 10  10      45222 non-null  int64 
 11  11      45222 non-null  int64 
 12  12      45222 non-null  int64 
 13  13      45222 non-null  object
 14  14      45222 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


In [8]:
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
adult_data.columns = columns

In [9]:
adult_data.nunique()

age                  74
workclass             7
fnlwgt            26741
education            16
education-num        16
marital-status        7
occupation           14
relationship          6
race                  5
sex                   2
capital-gain        121
capital-loss         97
hours-per-week       96
native-country       41
income                4
dtype: int64

In [10]:
print("age",adult_data['age'].unique())
print("workclass",adult_data['workclass'].unique())
print("fnlwgt",adult_data['fnlwgt'].unique())
print("education",adult_data['education'].unique())
print("education-num",adult_data['education-num'].unique())
print("marital-status",adult_data['marital-status'].unique())
print("occupation",adult_data['occupation'].unique())
print("relationship",adult_data['relationship'].unique())
print("race",adult_data['race'].unique())
print("sex",adult_data['sex'].unique())
print("capital-gain",adult_data['capital-gain'].unique())
print("capital-loss",adult_data['capital-loss'].unique())
print("hours-per-week",adult_data['hours-per-week'].unique())
print("native-country",adult_data['native-country'].unique())
print("income",adult_data['income'].unique())

age [39 50 38 53 28 37 49 52 31 42 30 23 32 34 25 43 40 54 35 59 56 19 20 45
 22 48 21 24 57 44 41 29 47 46 36 79 27 18 33 76 55 61 70 64 71 66 51 58
 26 17 60 90 75 65 77 62 63 67 74 72 69 68 73 81 78 88 80 84 83 85 82 86
 89 87]
workclass [' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' Self-emp-inc' ' Without-pay']
fnlwgt [ 77516  83311 215646 ... 173449  89686 350977]
education [' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' 7th-8th' ' Doctorate' ' Assoc-voc' ' Prof-school'
 ' 5th-6th' ' 10th' ' Preschool' ' 12th' ' 1st-4th']
education-num [13  9  7 14  5 10 12  4 16 11 15  3  6  1  8  2]
marital-status [' Never-married' ' Married-civ-spouse' ' Divorced'
 ' Married-spouse-absent' ' Separated' ' Married-AF-spouse' ' Widowed']
occupation [' Adm-clerical' ' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Sales' ' Transport-moving' ' Farming-fishing'
 ' Machine-op-inspct' ' Tech-support' ' Craft-

In [11]:
adult_data['income'] = adult_data['income'].astype(str).str.rstrip('.')
print("income",adult_data['income'].unique())

income [' <=50K' ' >50K']


In [12]:
# Drop unnecessary columns:
# 'education' is redundant because 'education-num' already encodes it numerically.
# 'fnlwgt' represents sampling weights and does not contribute to feature relationships.

adult_data = adult_data.drop(columns=['education', 'fnlwgt'])

In [13]:
adult_data.head(5)

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [14]:
categorical_columns = [
    'workclass', 
    'marital-status', 
    'occupation', 
    'relationship', 
    'race', 
    'sex', 
    'native-country', 
    'income'
] 
for col in categorical_columns:
    encoder = LabelEncoder()
    not_null_idx = adult_data[col].notnull()
    adult_data.loc[not_null_idx, col] = encoder.fit_transform(adult_data.loc[not_null_idx, col])

In [15]:
adult_data.head(5)

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,5,13,4,0,1,4,1,2174,0,40,38,0
1,50,4,13,2,3,0,4,1,0,0,13,38,0
2,38,2,9,0,5,1,4,1,0,0,40,38,0
3,53,2,7,2,5,0,2,1,0,0,40,38,0
4,28,2,13,2,9,5,2,0,0,0,40,4,0


In [16]:
# List of (X_index, Y_index, Z_index) triplets for CMI calculation
# Column indices after dropping 'education' and 'fnlwgt':

# 0: age
# 1: workclass
# 2: education-num
# 3: marital-status
# 4: occupation
# 5: relationship
# 6: race
# 7: sex
# 8: capital-gain
# 9: capital-loss
# 10: hours-per-week
# 11: native-country
# 12: income

cmi_triplets = [
    (0, 12, 2),  # (age, income, education-num)
    (6, 12, 2),  # (race, income, education-num)
    (7, 12, 2),  # (sex, income, education-num)
    (11, 12, 2), # (native-country, income, education-num)
    (0, 10, 4),  # (age, hours-per-week, occupation)
    (6, 10, 4),  # (race, hours-per-week, occupation)
    (7, 10, 3),  # (sex, hours-per-week, marital-status)
    (11, 10, 4), # (native-country, hours-per-week, occupation)
]


In [17]:
# continuous and discrete column indices
discrete_columns = [1, 3, 4, 5, 6, 7, 11, 12]
continuous_columns = [0, 2, 8, 9, 10]

# Create a copy of the original data to preserve the raw values
adult_data_scaled = adult_data.copy()

# Normalize only the continuous columns using MinMaxScaler
scaler = MinMaxScaler()
adult_data_scaled.iloc[:, continuous_columns] = scaler.fit_transform(adult_data_scaled.iloc[:, continuous_columns])

In [18]:
# Sampling strategy (Test vs Full)

#adult_data_sampled = adult_data_scaled

sample_indices = np.random.choice(adult_data_scaled.shape[0], size=100, replace=False)

adult_data_sampled = adult_data_scaled.iloc[sample_indices]
data_np = adult_data_sampled.values.astype(float)
data_tensor= torch.tensor(data_np,dtype=torch.float32)

In [19]:
#Encode and combine data

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
one_hot_encoded = encoder.fit_transform(adult_data_sampled.iloc[:, discrete_columns])
continuous_part = adult_data_sampled.iloc[:, continuous_columns].values
data_processed = np.concatenate([one_hot_encoded, continuous_part], axis=1)

one_hot_encoded_tensor = torch.tensor(one_hot_encoded, dtype=torch.float32)
continuous_part_tensor = torch.tensor(continuous_part, dtype=torch.float32)
data_processed_tensor = torch.cat([one_hot_encoded_tensor, continuous_part_tensor], dim=1)


In [20]:
# GRID SERACH
#BOOTSTRAP

In [26]:
sys.path.append(r"C:\Users\admin\Desktop\Imputation_Fairness") 

import Utils
from Inject_Missing_Values import Inject_Missing_Values
import RR_imputer
import Sinkhorn_CMI
import SinkhornImputation
import SoftImpute
from CMI_torch import compute_all_cmi_methods, estimate_CMI_soft_kronecker_gaussian, estimate_CMI_gumbel_softmax_kernel, \
    estimate_CMI_separate_kernel


In [22]:
for triplet in cmi_triplets:
    cmi1, cmi2, cmi3 = compute_all_cmi_methods(data_tensor, data_processed_tensor, triplet, encoder, discrete_columns, continuous_columns
                                               )

    print(f"Triplet {triplet} :")
    print("  CMI (Soft Kronecker + Gaussian):", cmi1.item())
    print("  CMI (Gumbel-Softmax + Gaussian):", cmi2.item())
    print("  CMI (Gumbel on Discrete + Separate Kernels):", cmi3.item())
    print()

Triplet (0, 12, 2) :
  CMI (Soft Kronecker + Gaussian): 0.0988466739654541
  CMI (Gumbel-Softmax + Gaussian): 0.09884679317474365
  CMI (Gumbel on Discrete + Separate Kernels): 0.09884703159332275

Triplet (6, 12, 2) :
  CMI (Soft Kronecker + Gaussian): 0.03047013282775879
  CMI (Gumbel-Softmax + Gaussian): 0.030469775199890137
  CMI (Gumbel on Discrete + Separate Kernels): 0.030470967292785645

Triplet (7, 12, 2) :
  CMI (Soft Kronecker + Gaussian): 0.027495622634887695
  CMI (Gumbel-Softmax + Gaussian): 0.02749502658843994
  CMI (Gumbel on Discrete + Separate Kernels): 0.027495741844177246

Triplet (11, 12, 2) :
  CMI (Soft Kronecker + Gaussian): 0.02697741985321045
  CMI (Gumbel-Softmax + Gaussian): 0.026976823806762695
  CMI (Gumbel on Discrete + Separate Kernels): 0.02697741985321045

Triplet (0, 10, 4) :
  CMI (Soft Kronecker + Gaussian): 0.11853170394897461
  CMI (Gumbel-Softmax + Gaussian): 0.11852908134460449
  CMI (Gumbel on Discrete + Separate Kernels): 0.1185312271118164

T

# Injection of 25% MCAR Missingness

In [27]:
# Step 1: Split the full dataset into features (X) and target (Y)
X = adult_data_sampled.iloc[:, :-1]
Y = adult_data_sampled.iloc[:, -1]

# Step 2: Inject 25% MCAR missingness into the feature set X
generator_mcar25 = Inject_Missing_Values()
X_miss_mcar25, index_mcar25 = generator_mcar25.MCAR(X, missing_rate=25)

# Step 3: Report total missing percentage
total_missing_percentage = X_miss_mcar25.isnull().sum().sum() / X_miss_mcar25.size * 100
print(f"Total Missing Percentage (MCAR 25%): {total_missing_percentage:.2f}%")

# Step 4: Report missing percentage per column
missing_per_column = (X_miss_mcar25.isnull().sum() / len(X_miss_mcar25)) * 100
print(missing_per_column)

Total Missing Percentage (MCAR 25%): 25.00%
age               20.0
workclass         32.0
education-num     19.0
marital-status    29.0
occupation        26.0
relationship      21.0
race              24.0
sex               19.0
capital-gain      28.0
capital-loss      28.0
hours-per-week    24.0
native-country    30.0
dtype: float64


In [28]:
X_miss_with_Y = X_miss_mcar25.copy()
X_miss_with_Y["target"] = Y

X_numpy = X.to_numpy(dtype=float)
X_tensor = torch.tensor(X_numpy, dtype=torch.float64)

Y_numpy = Y.to_numpy(dtype=int)
Y_tensor = torch.tensor(Y_numpy, dtype=torch.int32)
X_tensor_with_Y = torch.cat([X_tensor, Y_tensor.reshape(-1, 1)], dim=1)

X_miss_mcar25_numpy = X_miss_mcar25.to_numpy(dtype=float)
X_miss_mcar25_tensor = torch.tensor(X_miss_mcar25_numpy, dtype=torch.float64)
missing_value_mask = torch.isnan(X_miss_mcar25_tensor).double()

X_miss_with_Y_numpy = X_miss_with_Y.to_numpy(dtype=float)
X_miss_with_Y_tensor = torch.tensor(X_miss_with_Y_numpy, dtype=torch.float64)
missing_value_mask_with_Y = torch.isnan(X_miss_with_Y_tensor).double()



In [29]:
# Sinkhorn

def impute_sinkhorn():
    batchsize = 128
    lr = 1e-2
    epsilon = pick_epsilon(X_miss_mcar25_tensor)  # Determines Sinkhorn regularization strength
    sinkhorn_imputer = SinkhornImputation(eps=epsilon, batchsize=batchsize, lr=lr, niter=iterations)
    sinkhorn_filled_tensor, maes_mcar25, rmses_mcar25 = sinkhorn_imputer.fit_transform(
        X_miss_mcar25_tensor,
        verbose=True,
        report_interval=50,
        X_true=X_tensor,  # for tracking MAE/RMSE during training
    )
    return sinkhorn_filled_tensor


In [30]:
# Sinkhorn_CMI
def impute_sinkhorn_cmi(train_triplet_index, train_cmi, highest_lambda_cmi):
    # Use train_triplet_index=-1 for all triplets
    batchsize = 128
    lr = 1e-2
    epsilon = pick_epsilon(X_miss_with_Y_tensor)
    sk_imputer = SinkhornImputation_CMI(
        eps=epsilon,
        batchsize=batchsize,
        lr=lr,
        niter=iterations,
        highest_lamda_cmi=highest_lambda_cmi,
        cmi_index=train_cmi,
    )
    sk_imp_mcar25, maes_mcar25, rmses_mcar25, history_mcar25 = sk_imputer.fit_transform(
        X_miss_with_Y_tensor,
        Y=Y_tensor if train_cmi in [1, 2] else None,
        data_processed=data_processed_tensor,
        one_hot_encoded=one_hot_encoded_tensor,
        continuous_part=continuous_part_tensor,
        verbose=True,
        report_interval=50,
        X_true=torch.cat([X_tensor, Y_tensor.reshape(-1, 1)], dim=1),  # ground truth data
        X_cols=[cmi_triplets[train_triplet_index][0]] if train_triplet_index != -1 else [t[0] for t in cmi_triplets],
        Y_cols=[cmi_triplets[train_triplet_index][1]] if train_triplet_index != -1 else [t[1] for t in cmi_triplets],
        Z_cols=[cmi_triplets[train_triplet_index][2]] if train_triplet_index != -1 else [t[2] for t in cmi_triplets],
        encoder=encoder,
        discrete_columns=discrete_columns,
        continuous_columns=continuous_columns
    )

    return sk_imp_mcar25


In [31]:
# Mean Imputation

def impute_mean():
    mean_imp_mcar25 = SimpleImputer().fit_transform(X_miss_mcar25_tensor.cpu())
    return torch.Tensor(mean_imp_mcar25).to(device)


In [32]:
# Imputation by Chained Equations
def impute_mice():
    from sklearn.experimental import enable_iterative_imputer
    from sklearn.impute import IterativeImputer

    ice_imp_mcar25 = IterativeImputer(random_state=0, max_iter=iterations).fit_transform(X_miss_mcar25_tensor.cpu())
    ice_imp_mcar25_torch = torch.tensor(ice_imp_mcar25).to(device)
    return ice_imp_mcar25_torch

In [33]:
# Soft Imputation
def impute_soft():
    from SoftImpute import cv_softimpute, softimpute

    cv_error_mcar25, grid_lambda_mcar25 = cv_softimpute(X_miss_mcar25_numpy, grid_len=15)
    lbda_mcar25 = grid_lambda_mcar25[np.argmin(cv_error_mcar25)]
    soft_imp_mcar25 = softimpute((X_miss_mcar25_numpy), lbda_mcar25)[1]
    soft_imp_mcar25_torch = torch.tensor(soft_imp_mcar25).to(device)
    return soft_imp_mcar25_torch

In [34]:
def make_processed_data(X_with_y, discrete_columns, continuous_columns):
    data_continuous = X_with_y[:, continuous_columns]
    discrete_distributions = generate_distributions_for_discrete_data(X_with_y, discrete_columns, encoder)
    data_combined_tensor = torch.cat([discrete_distributions, data_continuous], dim=1)
    return data_combined_tensor

In [35]:
def evaluate_imputation(imputed_tensor, missing_value_mask, ground_truth, discrete_columns, continuous_columns,
                        meta_data={}):
    # e.g, meta_data = {'imputation_method': 'sk', train_triplet: 0}
    results = []
    with torch.no_grad():
        final_mae = MAE(imputed_tensor, ground_truth, missing_value_mask).item()
        final_rmse = RMSE(imputed_tensor, ground_truth, missing_value_mask).item()
        print(f"Final MAE: {final_mae:.4f}")
        print(f"Final RMSE: {final_rmse:.4f}")
        Y_tensor_reshaped = Y_tensor.view(-1, 1)
        X_with_y = torch.cat([imputed_tensor, Y_tensor_reshaped], dim=1)
        # Estimate (CMI) after imputation
        # for each predefined triplet using all three CMI estimation methods
        for test_triplet_index, triplet in enumerate(cmi_triplets):
            processed_tensor = make_processed_data(X_with_y, discrete_columns, continuous_columns)
            cmi1, cmi2, cmi3 = compute_all_cmi_methods(X_with_y,
                                                       processed_tensor,  # One-hot encoded + scaled continuous
                                                       triplet,  # (X, Y, Z)
                                                       encoder,
                                                       discrete_columns,
                                                       continuous_columns
                                                       )
            print(f"[{meta_data.get('imputation_method', 'Undefined Method')}] Triplet {triplet} :")
            print(" CMI (Soft Kronecker + Gaussian):", f"{cmi1.item():.4f}")
            print(" CMI (Gumbel-Softmax + Gaussian):", f"{cmi2.item():.4f}")
            print(" CMI (Gumbel on Discrete + Separate Kernels):", f"{cmi3.item():.4f}")
            print()
            for cmi_idx, cmi in enumerate([cmi1, cmi2, cmi3]):
                results.append({
                    **meta_data,
                    'test_triplet': test_triplet_index + 1,
                    'MAE': final_mae,
                    'RMSE': final_rmse,
                    'cmi_method': f'method-{cmi_idx + 1}',
                    'cmi': cmi.item(),
                })
    results_df = pd.DataFrame(results)
    return results_df